In [1]:
import os
import shutil
import random

def split_dataset(dataset_dir, train_dir, val_dir, split_ratio=0.8, seed=42):
    random.seed(seed)
    # Create train and validation directories if they don't exist
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)

    # Iterate through each class subfolder
    for class_name in os.listdir(dataset_dir):
        class_dir = os.path.join(dataset_dir, class_name)
        if os.path.isdir(class_dir):
            # Create class subfolders in train and validation directories
            train_class_dir = os.path.join(train_dir, class_name)
            val_class_dir = os.path.join(val_dir, class_name)
            os.makedirs(train_class_dir, exist_ok=True)
            os.makedirs(val_class_dir, exist_ok=True)

            # List all images in the class subfolder
            images = [f for f in os.listdir(class_dir) if os.path.isfile(os.path.join(class_dir, f))]

            # Shuffle the list of images
            random.shuffle(images)

            # Calculate the number of images for training and validation
            num_train = int(len(images) * split_ratio)
            train_images = images[:num_train]
            val_images = images[num_train:]

            # Copy images to train and validation directories
            for img in train_images:
                src = os.path.join(class_dir, img)
                dst = os.path.join(train_class_dir, img)
                shutil.copyfile(src, dst)

            for img in val_images:
                src = os.path.join(class_dir, img)
                dst = os.path.join(val_class_dir, img)
                shutil.copyfile(src, dst)

# Set paths
dataset_dir = 'dataset'
train_dir = 'animals/train'
val_dir = 'animals/validation'

# Split dataset
split_dataset(dataset_dir, train_dir, val_dir, split_ratio=0.8, seed=42)


In [1]:
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.transforms import functional as F
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision.models.detection import fasterrcnn_resnet50_fpn

# Define your custom dataset class
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.dataset = ImageFolder(root_dir)
        self.classes = self.dataset.classes  # Extract class labels from ImageFolder dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        img, label = self.dataset[idx]
        if self.transform:
            img = self.transform(img)
        return img, label

# Define transformations for data augmentation and normalization
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224, 224)),  # Resize images to fit pre-trained model input
    torchvision.transforms.ToTensor(),           # Convert PIL image to tensor
    torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),  # Normalize images
])

# Define train and validation datasets
print("Creating train and validation datasets...")
train_dataset = CustomDataset(root_dir='animals/train', transform=transform)
val_dataset = CustomDataset(root_dir='animals/validation', transform=transform)

# Define data loaders
print("Creating data loaders...")
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=4)

# Load pre-trained Faster R-CNN model
print("Loading pre-trained Faster R-CNN model...")
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# Replace the classifier with a new one for finetuning
num_classes = len(train_dataset.classes)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# Define device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move model to device
model.to(device)

# Define optimizer and loss function
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

# Define training function
def train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq):
    print(f"Epoch [{epoch+1}/{num_epochs}], Training...")
    model.train()
    for images, targets in data_loader:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

# Define validation function
@torch.no_grad()
def evaluate(model, data_loader, device):
    print("Validating...")
    model.eval()
    running_loss = 0.0
    for images, targets in data_loader:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        running_loss += losses.item()

    return running_loss / len(data_loader)

# Train the model
print("Training started...")
num_epochs = 10
print_freq = 10
for epoch in range(num_epochs):
    train_one_epoch(model, optimizer, train_loader, device, epoch, print_freq)
    lr_scheduler.step()
    val_loss = evaluate(model, val_loader, device)
    print(f"Epoch [{epoch+1}/{num_epochs}], Validation Loss: {val_loss}")

# Save the trained model
torch.save(model.state_dict(), 'trained_model.pth')
print("Training completed. Model saved.")

Creating train and validation datasets...
Creating data loaders...
Loading pre-trained Faster R-CNN model...


d:\Projects\Object_Detection\streamlit_object_detection_webapp\venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Projects\Object_Detection\streamlit_object_detection_webapp\venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Training started...
Epoch [1/10], Training...
